# Методы исследования характеристических свойств нейронных сетей с применением теоретико-игрового подхода

- **Теория**: И.В.Томилов ivan-tomilov3@yandex.ru
- **Реализация**: М.А.Зайцева maria@tail-call.ru
- **Поддержка CUDA**: А.Е.Григорьева admin@linkennt.ru
- **Ревизия**: 6

- **Другие ревизии**: [ссылка](https://disk.yandex.ru/d/aZozDpBlzh_z1A)
<!-- please do not append text into this block -->

## 0. Изменения

* Код разделён на библиотеку + блокнот для облегчения разработки
* За ненадобностью отключены тренировка и валидация RegularNetwork
* .backward():
```python
bernoulli_mask = torch.bernoulli(torch.ones(grad_input.size(1))) # было 0
```
* .backward():
```python
grad_input = grad_input @ diagonal_mask # было diagonal_mask @ grad_input
```
* Добавлены приближенные версии для некоторых графиков

## 1. Настройки

In [ ]:
## 1.1 Install dependencies

# pip install -r requirements.txt

In [ ]:
## 1.2 Import common library

import cgtnnlib.common as common

## 2. Обучение

In [ ]:

# 5.3. Training

import matplotlib.pyplot as plt
import torch

common.train_main()
print("Done!")

In [ ]:
common.report.save()

## 3. Валидация 

In [ ]:

# 6.2. Evaluation

from dataclasses import dataclass
from typing import Literal

from cgtnnlib.AugmentedReLUNetwork import AugmentedReLUNetwork
from cgtnnlib.EvaluationSubplots import EvaluationSubplots
from cgtnnlib.LearningTask import LearningTask, is_classification_task, is_regression_task, classification_task, regression_task
from IPython.display import clear_output

@dataclass
class EvaluationParameters:
    dataset: common.Dataset
    model_path: str
    is_binary_classification: bool
    is_regression: bool
    inputs_count: int
    outputs_count: int
    task: LearningTask
    experiment_parameters: common.ExperimentParameters
    report_key: str

def model_path_for(
    model_a_or_b: Literal["A", "B"],
    dataset: common.Dataset
):
    if model_a_or_b == "A":
        return dataset.model_a_path(experiment_params)
    elif model_a_or_b == "B":
        return dataset.model_b_path(experiment_params)
    else:
        raise TypeError('model_a_or_b must be A or B')

def evaluate(
    model_a_or_b: Literal["A", "B"],
    constructor: type,
    experiment_params: EvaluationParameters
):
    """
    Оценивает модель `"A"` (`RegularNetwork`) или `"B"`
    (`AugmentedReLUNetwork`) согласно параметрам `experiment_params` на
    наборах данных из `DATASETS`.
    Рисует графики метрик и сохраняет их на диск.
    
    - `constructor` может быть `RegularNetwork` или `AugmentedReLUNetwork`
      и должен соответствовать переданному `model_a_or_b`.
    """
    

    eval_params_items: list[EvaluationParameters] = [
        EvaluationParameters(
            dataset=common.DATASETS[0],
            model_path=model_path_for(model_a_or_b, common.DATASETS[0]),
            is_binary_classification=True,
            is_regression=False,
            inputs_count=30,
            outputs_count=2,
            task=classification_task,
            experiment_parameters=experiment_params,
            report_key=f'evaluate_{constructor.__name__}_{common.DATASETS[0].number}_p{experiment_params.p}_N{experiment_params.iteration}',
        ),
        EvaluationParameters(
            dataset=common.DATASETS[1],
            model_path=model_path_for(model_a_or_b, common.DATASETS[1]),
            is_binary_classification=False,
            is_regression=False,
            inputs_count=6,
            outputs_count=4,
            task=classification_task,
            experiment_parameters=experiment_params,
            report_key=f'evaluate_{constructor.__name__}_{common.DATASETS[1].number}_p{experiment_params.p}_N{experiment_params.iteration}',
        ),
        EvaluationParameters(
            dataset=common.DATASETS[2],
            model_path=model_path_for(model_a_or_b, common.DATASETS[2]),
            is_binary_classification=False,
            is_regression=True,
            inputs_count=19,
            outputs_count=1,
            task=regression_task,
            experiment_parameters=experiment_params,
            report_key=f'evaluate_{constructor.__name__}_{common.DATASETS[2].number}_p{experiment_params.p}_N{experiment_params.iteration}',
        ),
    ]

    fig, axs = plt.subplots(3, len(eval_params_items), figsize=(10, 12))
    fig.set_size_inches(35, 20)

    for (i, eval_params) in enumerate(eval_params_items):
        evaluated_model = constructor(
            inputs_count=eval_params.inputs_count,
            outputs_count=eval_params.outputs_count,
            p=experiment_params.p
        )

        clear_output(wait=True)
        print(f'Evaluating model at {eval_params.model_path}...')
        evaluated_model.load_state_dict(torch.load(eval_params.model_path))

        plot_title = f'Evaluating {eval_params.model_path}: p = {eval_params.experiment_parameters.p}, N = {eval_params.experiment_parameters.iteration}'

        subplots = EvaluationSubplots(
            accuracy_ax=axs[0, i],
            f1_ax=axs[1, i],
            roc_auc_ax=axs[2, i],
            mse_ax=axs[0, i],
            r2_ax=axs[1, i],
        )
    
        if is_classification_task(eval_params.task):
            common.plot_evaluation_of_classification(
                df=common.evaluate_classification_model(
                    evaluated_model=evaluated_model,
                    dataset=eval_params.dataset,
                    report_key=eval_params.report_key,
                    is_binary_classification=eval_params.is_binary_classification
                ),
                accuracy_ax=subplots.accuracy_ax,
                f1_ax=subplots.f1_ax,
                roc_auc_ax=subplots.roc_auc_ax,
                title=plot_title
            )
        elif is_regression_task(eval_params.task):
            common.plot_evaluation_of_regression(
                df=common.evaluate_regression_model(
                    evaluated_model=evaluated_model,
                    dataset=eval_params.dataset,
                    report_key=eval_params.report_key
                ),
                mse_ax=subplots.mse_ax,
                r2_ax=subplots.r2_ax,
                title=plot_title
            )
        else:
            raise ValueError(f"Unknown task: {eval_params.task}")

    # lib.save_plot(f'evaluate_{model_a_or_b}_p{experiment_params.p}_N{experiment_params.iteration}')


for experiment_params in common.iterate_experiment_parameters():
    # evaluate(
    #     model_a_or_b='A',
    #     constructor=RegularNetwork,
    #     experiment_params=experiment_params
    # )
    evaluate(
        model_a_or_b='B',
        constructor=AugmentedReLUNetwork,
        experiment_params=experiment_params
    )

common.report.save()
print("Done!")

## 7. Анализ данных

In [1]:
# 1. Загрузка отчёта

import json

with open('./report/report.json') as fd:
    report = json.load(fd)

In [2]:
# 2. Составление индекса для поиска по отчёту

import pandas as pd

df = pd.DataFrame([[key] + key.split('_') for key in report.keys()])
df.columns = ['Key', 'Measurement', 'Network', 'Dataset', 'P', 'N']
df = df[df['Key'] != 'started']
df = df[df['Key'] != 'saved']
df.Dataset = df.Dataset.apply(lambda x: int(x))
df.P = df.P.apply(lambda x: float(x[1:]))
df.N = df.N.apply(lambda x: int(x[1:]))
df

,Key,Measurement,Network,Dataset,P,N
1,loss_AugmentedReLUNetwork_1_p0_N0,loss,AugmentedReLUNetwork,1,0.00,0
2,loss_AugmentedReLUNetwork_2_p0_N0,loss,AugmentedReLUNetwork,2,0.00,0
3,loss_AugmentedReLUNetwork_3_p0_N0,loss,AugmentedReLUNetwork,3,0.00,0
4,loss_AugmentedReLUNetwork_1_p0.01_N0,loss,AugmentedReLUNetwork,1,0.01,0
5,loss_AugmentedReLUNetwork_2_p0.01_N0,loss,AugmentedReLUNetwork,2,0.01,0
...,...,...,...,...,...,...
450,evaluate_AugmentedReLUNetwork_2_p0.95_N9,evaluate,AugmentedReLUNetwork,2,0.95,9
451,evaluate_AugmentedReLUNetwork_3_p0.95_N9,evaluate,AugmentedReLUNetwork,3,0.95,9
452,evaluate_AugmentedReLUNetwork_1_p0.99_N9,evaluate,AugmentedReLUNetwork,1,0.99,9
453,evaluate_AugmentedReLUNetwork_2_p0.99_N9,evaluate,AugmentedReLUNetwork,2,0.99,9


In [6]:
def df_head_fraction(df: pd.DataFrame, frac: float) -> pd.DataFrame:
    n_rows = int(len(df) * frac)
    return df.head(n_rows)

df_head_fraction(df, frac=0.15)

,Key,Measurement,Network,Dataset,P,N
1,loss_AugmentedReLUNetwork_1_p0_N0,loss,AugmentedReLUNetwork,1,0.00,0
2,loss_AugmentedReLUNetwork_2_p0_N0,loss,AugmentedReLUNetwork,2,0.00,0
3,loss_AugmentedReLUNetwork_3_p0_N0,loss,AugmentedReLUNetwork,3,0.00,0
4,loss_AugmentedReLUNetwork_1_p0.01_N0,loss,AugmentedReLUNetwork,1,0.01,0
5,loss_AugmentedReLUNetwork_2_p0.01_N0,loss,AugmentedReLUNetwork,2,0.01,0
...,...,...,...,...,...,...
63,loss_AugmentedReLUNetwork_3_p0.99_N2,loss,AugmentedReLUNetwork,3,0.99,2
64,loss_AugmentedReLUNetwork_1_p0_N3,loss,AugmentedReLUNetwork,1,0.00,3
65,loss_AugmentedReLUNetwork_2_p0_N3,loss,AugmentedReLUNetwork,2,0.00,3
66,loss_AugmentedReLUNetwork_3_p0_N3,loss,AugmentedReLUNetwork,3,0.00,3


In [16]:
# 3. Вывод графиков

import os
from dataclasses import dataclass

import matplotlib.pyplot as plt
import pandas as pd

@dataclass
class PlotParams:
    measurement: str
    dataset_number: int
    network = 'AugmentedReLUNetwork'
    metric: str
    p: int
    
def compute_dataframe(plot_params: PlotParams) -> pd.DataFrame:
    rows = (
        df
            .loc[df.Measurement == plot_params.measurement]
            .loc[df.Dataset == plot_params.dataset_number]
            .loc[df.Network == plot_params.network]
            .loc[df.P == plot_params.p]
    )

    if plot_params.measurement == 'loss':
        values = pd.DataFrame([report[row.Key] for row in rows.itertuples()])
    else:
        cols = []
        
        for row in rows.itertuples():
            report_data = report[row.Key]
            cols.append(report_data[plot_params.metric])
            
        values = pd.DataFrame(cols)

    result = values.quantile([0.25, 0.75]).transpose()
    result['mean'] = values.mean()
    return result


def plot_curve(
    ax: object,
    means: pd.Series,
    lowerqs: pd.Series,
    upperqs: pd.Series,
    zmeans: pd.Series,
    zlowerqs: pd.Series,
    zupperqs: pd.Series,
    X: pd.Series,
    title: str,
    xlabel: str,
    ylabel: str,
):
    ax.plot(X, zmeans, label='Mean of p = 0', color='lightblue')
    ax.fill_between(X, zlowerqs, zupperqs, color='lightgray', alpha=0.5, label='0.25 to 0.75 Quantiles, p = 0')
    ax.plot(X, means, label='Mean', color='blue')
    ax.fill_between(X, lowerqs, upperqs, color='gray', alpha=0.5, label='0.25 to 0.75 Quantiles')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend()


for (measurement, dataset_number, xlabel, frac) in [
    ('loss', 1, 'iteration', 1),
    ('loss', 1, 'iteration', 0.02),
    ('loss', 1, 'iteration', 0.1),
    ('loss', 1, 'iteration', 0.2),
    ('evaluate', 1, 'noise factor', 1),
    ('loss', 2, 'iteration', 1),
    ('loss', 2, 'iteration', 0.02),
    ('loss', 2, 'iteration', 0.1),
    ('loss', 2, 'iteration', 0.2),
    ('evaluate', 2, 'noise factor', 1),
    ('loss', 3, 'iteration', 1),
    ('loss', 3, 'iteration', 0.02),
    ('loss', 3, 'iteration', 0.1),
    ('loss', 3, 'iteration', 0.2),
    ('evaluate', 3, 'noise factor', 1),
]:
    if measurement == 'loss':
        metrics = ['loss']
    else:
        if dataset_number == 3:
            metrics = ['r2', 'mse']
        else:
            metrics = ['f1', 'accuracy', 'roc_auc']
    
    fig, axs = plt.subplots(len(metrics), 6, figsize=(24, len(metrics) * 6))

    for (i, metric) in enumerate(metrics):
        def make_curve(p: float) -> pd.DataFrame:
            return compute_dataframe(plot_params=PlotParams(
                measurement=measurement,
                dataset_number=dataset_number,
                metric=metric,
                p=p,
            ))

        reference_curve: pd.DataFrame = df_head_fraction(
            df=make_curve(p=0),
            frac=frac
        )

        for (j, p) in enumerate([0.01, 0.05, 0.5, 0.9, 0.95, 0.99]):
            curve: pd.DataFrame = df_head_fraction(
                df=make_curve(p=p),
                frac=frac
            )

            plot_curve(
                ax=axs[i, j] if len(metrics) > 1 else axs[j],
                means=curve['mean'],
                lowerqs=curve[0.25],
                upperqs=curve[0.75],
                zmeans=reference_curve['mean'],
                zlowerqs=reference_curve[0.25],
                zupperqs=reference_curve[0.75],
                X=curve.index,
                title=f'p = {p}',
                xlabel=xlabel,
                ylabel=metric,
            )
    fig.suptitle(f'Dataset #{dataset_number}, zoom factor: {frac}')
    plt.tight_layout()
    path = os.path.join('report/', f'{measurement}_{dataset_number}_f{frac:.02f}.png')
    plt.savefig(path)
    plt.close()